Gene Expression Data Preprocessing

Overview:
This notebook performs normalization, filtering, and statistical analysis of gene expression data
from single-cell RNA sequencing (scRNA-seq).

The workflow involves:
1. Loading and cleaning the gene expression data from a CSV file.
2. Normalizing the data by calculating the gene expression ratio for each gene in each cell.
   (Each cell's gene expression value is divided by the total RNA detected for that cell.)
3. Computing key statistics for each gene, including the sum of gene expression, mean expression ratio,
   and variance.
4. Saving the results to a csv file in the 'gene_statistics' output dir for further analysis.

In [1]:
# Required Libraries
import os
import pandas as pd
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
import csv

In [2]:
# Function to load and clean data in chunks, allowing for large files to be processed
def load_and_clean_data(file_path, chunksize=1000):
    """
    Loads and cleans the data from a CSV file in chunks.
    It removes rows where all values are zero.
    
    Parameters:
    - file_path: str, the path to the CSV file
    - chunksize: int, the number of rows per chunk

    Returns:
    - data: DataFrame, the concatenated and cleaned data
    """
    chunks = []  # List to store chunks
    total_rows = sum(1 for _ in open(file_path)) - 1  # Calculate total rows minus header row
    
    # Read the data in chunks and remove rows where all values are zero
    try:
        with pd.read_csv(file_path, index_col=0, chunksize=chunksize) as reader:
            for chunk in tqdm(reader, desc="Reading and cleaning data", total=total_rows // chunksize):
                # Remove rows where all values are zero
                chunk = chunk.loc[~(chunk == 0).all(axis=1)]
                chunks.append(chunk)
    except pd.errors.ParserError as e:
        print(f"Error reading CSV file: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    
    # Concatenate the chunks into a single DataFrame
    data = pd.concat(chunks, axis=0)
    return data

In [3]:
# Function to normalize the gene expression data by dividing each gene expression by the column sum
def normalize_gene_expression(data):
    """
    Normalizes gene expression data by dividing each value by the sum of its column (i.e., total expression per gene).
    
    Parameters:
    - data: DataFrame, the gene expression data
    
    Returns:
    - normalized_data: DataFrame, the normalized data
    """
    column_sums = data.sum(axis=0)  # Sum of expression values for each gene
    normalized_data = data.copy()   # Create a copy to avoid modifying the original data
    
    # Normalize each column by dividing by its column sum
    for col in tqdm(data.columns, desc="Normalizing gene expression"):
        normalized_data[col] = data[col] / column_sums[col]
    
    return normalized_data

In [4]:
# # Function to filter out genes with low expression based on a threshold
# def filter_low_expression_genes(data, threshold=0.01):
#     """
#     Filters out genes with low expression across cells based on a threshold.
    
#     Parameters:
#     - data: DataFrame, the normalized gene expression data
#     - threshold: float, the minimum percentage of cells where a gene must be expressed to retain the gene
    
#     Returns:
#     - filtered_data: DataFrame, the filtered data
#     """
#     num_cells = data.shape[1]  # Number of cells (columns)
#     min_cells_expressed = threshold * num_cells  # Minimum number of cells required for a gene to be expressed
#     non_zero_counts = (data > 0).sum(axis=1)  # Count of non-zero values per gene (row)
    
#     # Use .loc to filter rows (genes) based on the condition
#     filtered_data = data.loc[non_zero_counts >= min_cells_expressed]
    
#     # Print the number of genes retained after filtering
#     print(f"Filtering complete: {filtered_data.shape[0]} genes retained out of {data.shape[0]} total.")
    
#     return filtered_data

In [5]:
# Function to compute gene statistics like sum, mean, and variance for each gene
def compute_gene_statistics(data):
    """
    Computes gene statistics such as sum, mean, and variance for each gene across all cells.
    
    Parameters:
    - data: DataFrame, the filtered gene expression data
    
    Returns:
    - gene_statistics: dict, a dictionary of gene statistics where keys are gene names
      and values are lists containing sum, mean, and variance for each gene
    """
    gene_statistics = {}  # Dictionary to store statistics
    
    # Function to calculate statistics for a single gene
    def calc_stats(gene):
        gene_values = data.loc[gene].values  # Get expression values for the gene
        gene_sum = gene_values.sum()         # Total expression across all cells
        mean_expression = gene_values.mean() # Mean expression value
        variance = gene_values.var()         # Variance of expression
        return gene, [gene_sum, mean_expression, variance]

    # Use tqdm to show progress bar while calculating gene statistics
    with ThreadPoolExecutor() as executor:
        results = tqdm(executor.map(calc_stats, data.index), total=len(data.index), desc="Calculating gene statistics")
        
        # Update the dictionary with calculated statistics
        for gene, stats in results:
            gene_statistics[gene] = stats

    return gene_statistics

In [6]:
# Function to save gene statistics to a CSV file
def save_dict_to_csv(gene_statistics, output_file):
    """
    Saves the gene statistics dictionary to a CSV file.
    
    Parameters:
    - gene_statistics: dict, the dictionary containing gene statistics
    - output_file: str, the name of the output CSV file
    
    Returns:
    - None
    """
    with open(output_file, 'w', newline='') as f:
        writer = csv.writer(f)
        # Write header row
        writer.writerow(["Gene", "Sum", "Mean", "Variance"])
        # Write gene statistics
        for gene, stats in gene_statistics.items():
            writer.writerow([gene] + stats)
    
    print(f"Gene statistics saved to '{output_file}'.")

In [7]:
# Recursive function to find 'dense-matrix.csv' in nested subdirectories
def find_dense_matrix_file(root_dir):
    """
    Recursively searches for the 'dense-matrix.csv' file in the given directory and its subdirectories.
    
    Parameters:
    - root_dir: str, the root directory to start the search
    
    Returns:
    - str or None: the full path to 'dense-matrix.csv' if found, otherwise None
    """
    for root, dirs, files in os.walk(root_dir):
        for file in files:
            if file == "dense-matrix.csv":
                return os.path.join(root, file)
    return None  # Return None if the file is not found

In [8]:
def process_gene_expression_data(file_path, threshold=0.01, chunksize=1000):
    """
    Processes the gene expression data by loading, cleaning, normalizing, filtering,
    and computing statistics for gene expression data.
    
    Parameters:
    - file_path: str, the path to the gene expression data file
    - threshold: float, the minimum percentage of cells where a gene must be expressed
    - chunksize: int, the number of rows to read per chunk when loading data
    
    Returns:
    - gene_statistics: dict, the dictionary of computed gene statistics
    """
    # Load and clean the data in chunks
    data = load_and_clean_data(file_path, chunksize=chunksize)
    
    # Normalize the gene expression data
    print("Normalizing gene expression data...")
    normalized_data = normalize_gene_expression(data)
    
    # Filter out low-expression genes based on the threshold
    # print("Filtering low-expression genes...")
    # filtered_data = filter_low_expression_genes(normalized_data, threshold)
    
    # Compute gene statistics for the filtered data
    gene_statistics = compute_gene_statistics(normalized_data)
    
    return gene_statistics

In [9]:
def process_all_files(input_dir, output_dir, threshold=0.01, chunksize=1000):
    """
    Loops through the input directory to find 'dense-matrix.csv' in each sample subdirectory,
    processes it, and saves the results in the output directory. Creates a .txt file for each missing
    dense-matrix.csv file. Stops checking further samples after encountering a missing sample directory.
    
    Parameters:
    - input_dir: str, the root directory containing cases and sample subdirectories
    - output_dir: str, the directory where the output CSV or text files will be saved
    - threshold: float, the minimum percentage of cells where a gene must be expressed
    - chunksize: int, the number of rows to read per chunk when loading data
    
    Returns:
    - None
    """
    # Ensure output directory exists, create it if not
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Sort case directories to ensure consistent processing order
    for case_dir in sorted(os.listdir(input_dir)):
        case_path = os.path.join(input_dir, case_dir)
        # Ensure it's a directory and ignore hidden/system files like .DS_Store
        if not os.path.isdir(case_path) or case_dir.startswith('.'):
            continue  # Skip non-directory or hidden/system files

        # Loop through sample directories within each case
        for sample_dir in sorted(os.listdir(case_path)):
            sample_path = os.path.join(case_path, sample_dir, 'single_cell')

            # Ignore hidden/system files in the sample directories as well
            if not os.path.isdir(os.path.join(case_path, sample_dir)) or sample_dir.startswith('.'):
                continue  # Skip non-directory or hidden/system files

            # Search for 'dense-matrix.csv' in the sample's subdirectories
            dense_matrix_path = find_dense_matrix_file(sample_path)

            # If dense-matrix.csv is found, process the file
            if dense_matrix_path:
                print(f"Processing {dense_matrix_path} for case {case_dir}, sample {sample_dir}...")
                try:
                    # Process the dense-matrix.csv file and compute statistics
                    gene_statistics = process_gene_expression_data(dense_matrix_path, threshold, chunksize)
                    
                    # Save the output as a CSV in the output directory
                    output_file = os.path.join(output_dir, f"{case_dir}_{sample_dir}.csv")
                    save_dict_to_csv(gene_statistics, output_file)
                except Exception as e:
                    print(f"Error processing file {dense_matrix_path}: {e}")
            
            # If dense-matrix.csv is not found, create a .txt file indicating the file is missing
            else:
                print(f"No dense-matrix.csv found for {case_dir}, sample {sample_dir}.")
                # Create a text file indicating no file was found
                output_file = os.path.join(output_dir, f"{case_dir}_{sample_dir}.txt")
                with open(output_file, 'w') as f:
                    f.write(f"No dense-matrix.csv found in {sample_path}")
                print(f"Created {output_file}.")

In [10]:
# Process the data and save results
try:
    # Input and output directories
    input_dir = "GDC-data"  # Input directory with all the cases
    output_dir = "gene_statistics"  # Output directory for results

    # Process all files
    process_all_files(input_dir, output_dir, threshold=0.01, chunksize=1000)

except Exception as e:
    print(f"An error occurred during processing: {e}")

Processing GDC-data/C3L-00359/1/single_cell/acfe95e5-b1ad-46ca-b709-46644c9a0c6d/qc_filtered_bc_feature_matrix/dense-matrix.csv for case C3L-00359, sample 1...


Reading and cleaning data: 61it [02:33,  2.52s/it]                        


Normalizing gene expression data...


Calculating gene statistics: 100%|██████████| 37387/37387 [19:36<00:00, 31.78it/s] 


Gene statistics saved to 'gene_statistics/C3L-00359_1.csv'.
Processing GDC-data/C3L-00606/1/single_cell/03845ba9-15a3-4216-b484-0489eb0fef90/qc_filtered_bc_feature_matrix/dense-matrix.csv for case C3L-00606, sample 1...


...

Gene statistics saved to 'gene_statistics/C3N-03186_1.csv'.
Processing GDC-data/C3N-03188/1/single_cell/f7404457-fae1-487d-8c35-345a457d2b30/qc_filtered_bc_feature_matrix/dense-matrix.csv for case C3N-03188, sample 1...


Reading and cleaning data: 61it [02:06,  2.07s/it]                        


Normalizing gene expression data...


Calculating gene statistics: 100%|██████████| 43011/43011 [24:44<00:00, 28.98it/s] 


Gene statistics saved to 'gene_statistics/C3N-03188_1.csv'.
